In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from math import sqrt
from tensorflow import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *


from preprocess import water_process
# from model.cnn import cnn_4layer, mlp_layer
from model.mlp import mlp_layer

2024-01-30 15:45:27.136774: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
dataset = pd.read_csv('../../data/Merged-update_hourly.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [4]:
np.percentile(data['MEAN_RAIN'], 95)

0.0053125

### Preprocess

In [5]:
# ====== preprocessing parameters ======
n_hours = 72
k = 24 
masked_value = 1e-10
split_1 = 0.7
split_2 = 0.85

In [7]:
train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = water_process(n_hours, k, masked_value, split_1, split_2)

Index(['MEAN_RAIN', 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2',
       'GATE_S26_1', 'GATE_S26_2', 'PUMP_S25B', 'PUMP_S26', 'HWS_S25A',
       'HWS_S25B', 'HWS_S26', 'WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26'],
      dtype='object')


In [8]:
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(67435, 96, 16) (14451, 96, 16) (14451, 96, 16) (67435, 96) (14451, 96) (14451, 96)


### Rain threshold

In [14]:
# train_X_mask_rain_acc = np.sum(train_X_mask[:, -24:, 0], axis=1)
# print(train_X_mask_rain_acc.shape)

# test_X_mask_rain_acc = np.sum(test_X_mask[:, -24:, 0], axis=1)
# print(test_X_mask_rain_acc.shape)

train_X_mask_rain_acc = train_X_mask[:, 0, 0]
print(train_X_mask_rain_acc.shape)

val_X_mask_rain_acc = val_X_mask[:, 0, 0]
print(val_X_mask_rain_acc.shape)

test_X_mask_rain_acc = test_X_mask[:, 0, 0]
print(test_X_mask_rain_acc.shape)

(67435,)
(14451,)
(14451,)


In [15]:
percentile = 95

merged_array = np.concatenate((train_X_mask_rain_acc, val_X_mask_rain_acc, test_X_mask_rain_acc))

percentile_rain = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_rain)

95th Percentile of Daily Rain: 0.010711610503493287


### train_X_mask_filter

In [16]:
# ======= extreme =======
train_X_mask_extreme = train_X_mask[train_X_mask_rain_acc > percentile_rain]
print(train_X_mask_extreme.shape)

train_ws_y_extreme = train_ws_y[train_X_mask_rain_acc > percentile_rain]
print(train_ws_y_extreme.shape)


# ======= normal =======
train_X_mask_normal = train_X_mask[train_X_mask_rain_acc <= percentile_rain]
print(train_X_mask_normal.shape)

train_ws_y_normal = train_ws_y[train_X_mask_rain_acc <= percentile_rain]
print(train_ws_y_normal.shape)

(3251, 96, 16)
(3251, 96)
(64184, 96, 16)
(64184, 96)


### val_X_mask_filter

In [17]:
# ======= extreme =======
val_X_mask_extreme = val_X_mask[val_X_mask_rain_acc > percentile_rain]
print(val_X_mask_extreme.shape)

val_ws_y_extreme = val_ws_y[val_X_mask_rain_acc > percentile_rain]
print(val_ws_y_extreme.shape)


# ======= normal =======
val_X_mask_normal = val_X_mask[val_X_mask_rain_acc <= percentile_rain]
print(val_X_mask_normal.shape)

val_ws_y_normal = val_ws_y[val_X_mask_rain_acc <= percentile_rain]
print(val_ws_y_normal.shape)

(536, 96, 16)
(536, 96)
(13915, 96, 16)
(13915, 96)


### test_X_mask_filter

In [18]:
# ======= extreme =======
test_X_mask_extreme = test_X_mask[test_X_mask_rain_acc > percentile_rain]
print(test_X_mask_extreme.shape)

test_ws_y_extreme = test_ws_y[test_X_mask_rain_acc > percentile_rain]
print(test_ws_y_extreme.shape)


# ======= normal =======
test_X_mask_normal = test_X_mask[test_X_mask_rain_acc <= percentile_rain]
print(test_X_mask_normal.shape)

test_ws_y_normal = test_ws_y[test_X_mask_rain_acc <= percentile_rain]
print(test_ws_y_normal.shape)

(1030, 96, 16)
(1030, 96)
(13421, 96, 16)
(13421, 96)


### Model

In [19]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0  # 0.1
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 100
EPOCHS = 1000
BATCH = 512
opt_num = k*4
input_shape = train_X_mask.shape[1:]

In [20]:
model = mlp_layer(input_shape=input_shape,
                   mlp_unit1=mlp_unit1,
                   mlp_unit2=mlp_unit2,
                   mlp_unit3=mlp_unit3,
                   mlp_unit4=mlp_unit4,
                   mlp_unit5=mlp_unit5,
                   mlp_unit6=mlp_unit6,
                   mlp_unit7=mlp_unit7,
                   mlp_unit8=mlp_unit8,
                   dropout=dropout,
                   masked_value=-1,
                   opt_num=opt_num
                  )
# model.summary()

In [21]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                          decay_steps=decay_steps,
                                                          decay_rate=decay_rate)

model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mse',
              metrics=['mae']
             )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint('../../saved_models/water_N_E_95.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True
                    )


model.fit([train_X_mask_normal], train_ws_y_normal,
          validation_data=([val_X_mask_normal], val_ws_y_normal),
          epochs=EPOCHS,
          batch_size=BATCH,
          verbose=2,
          shuffle=True,
          callbacks=[es, mc]
         )

Epoch 1/1000
126/126 - 6s - loss: 0.0436 - mae: 0.1633 - val_loss: 0.0238 - val_mae: 0.1304

Epoch 00001: val_mae improved from inf to 0.13040, saving model to ../../saved_models/water_N_E_95.h5
Epoch 2/1000
126/126 - 3s - loss: 0.0090 - mae: 0.0772 - val_loss: 0.0043 - val_mae: 0.0519

Epoch 00002: val_mae improved from 0.13040 to 0.05187, saving model to ../../saved_models/water_N_E_95.h5
Epoch 3/1000
126/126 - 3s - loss: 0.0016 - mae: 0.0301 - val_loss: 0.0032 - val_mae: 0.0451

Epoch 00003: val_mae improved from 0.05187 to 0.04509, saving model to ../../saved_models/water_N_E_95.h5
Epoch 4/1000
126/126 - 3s - loss: 0.0013 - mae: 0.0274 - val_loss: 0.0029 - val_mae: 0.0433

Epoch 00004: val_mae improved from 0.04509 to 0.04328, saving model to ../../saved_models/water_N_E_95.h5
Epoch 5/1000
126/126 - 3s - loss: 0.0011 - mae: 0.0248 - val_loss: 0.0027 - val_mae: 0.0414

Epoch 00005: val_mae improved from 0.04328 to 0.04143, saving model to ../../saved_models/water_N_E_95.h5
Epoch 6/1


Epoch 00047: val_mae improved from 0.02622 to 0.02565, saving model to ../../saved_models/water_N_E_95.h5
Epoch 48/1000
126/126 - 3s - loss: 1.5671e-04 - mae: 0.0078 - val_loss: 0.0010 - val_mae: 0.0255

Epoch 00048: val_mae improved from 0.02565 to 0.02550, saving model to ../../saved_models/water_N_E_95.h5
Epoch 49/1000
126/126 - 3s - loss: 1.5499e-04 - mae: 0.0077 - val_loss: 0.0010 - val_mae: 0.0254

Epoch 00049: val_mae improved from 0.02550 to 0.02544, saving model to ../../saved_models/water_N_E_95.h5
Epoch 50/1000
126/126 - 3s - loss: 1.5366e-04 - mae: 0.0077 - val_loss: 0.0010 - val_mae: 0.0253

Epoch 00050: val_mae improved from 0.02544 to 0.02534, saving model to ../../saved_models/water_N_E_95.h5
Epoch 51/1000
126/126 - 3s - loss: 1.5294e-04 - mae: 0.0077 - val_loss: 9.3537e-04 - val_mae: 0.0242

Epoch 00051: val_mae improved from 0.02534 to 0.02423, saving model to ../../saved_models/water_N_E_95.h5
Epoch 52/1000
126/126 - 3s - loss: 1.5046e-04 - mae: 0.0076 - val_loss: 9


Epoch 00095: val_mae did not improve from 0.02178
Epoch 96/1000
126/126 - 3s - loss: 1.0866e-04 - mae: 0.0063 - val_loss: 8.3171e-04 - val_mae: 0.0224

Epoch 00096: val_mae did not improve from 0.02178
Epoch 97/1000
126/126 - 3s - loss: 1.0835e-04 - mae: 0.0063 - val_loss: 7.9935e-04 - val_mae: 0.0221

Epoch 00097: val_mae did not improve from 0.02178
Epoch 98/1000
126/126 - 3s - loss: 1.0770e-04 - mae: 0.0063 - val_loss: 7.8663e-04 - val_mae: 0.0221

Epoch 00098: val_mae did not improve from 0.02178
Epoch 99/1000
126/126 - 3s - loss: 1.0724e-04 - mae: 0.0063 - val_loss: 8.0229e-04 - val_mae: 0.0222

Epoch 00099: val_mae did not improve from 0.02178
Epoch 100/1000
126/126 - 3s - loss: 1.0662e-04 - mae: 0.0063 - val_loss: 7.5297e-04 - val_mae: 0.0215

Epoch 00100: val_mae improved from 0.02178 to 0.02155, saving model to ../../saved_models/water_N_E_95.h5
Epoch 101/1000
126/126 - 3s - loss: 1.0642e-04 - mae: 0.0063 - val_loss: 8.1209e-04 - val_mae: 0.0222

Epoch 00101: val_mae did not 


Epoch 00145: val_mae did not improve from 0.02036
Epoch 146/1000
126/126 - 3s - loss: 9.0554e-05 - mae: 0.0056 - val_loss: 7.0681e-04 - val_mae: 0.0206

Epoch 00146: val_mae did not improve from 0.02036
Epoch 147/1000
126/126 - 3s - loss: 9.0615e-05 - mae: 0.0056 - val_loss: 6.8924e-04 - val_mae: 0.0204

Epoch 00147: val_mae did not improve from 0.02036
Epoch 148/1000
126/126 - 3s - loss: 9.0544e-05 - mae: 0.0056 - val_loss: 6.4520e-04 - val_mae: 0.0198

Epoch 00148: val_mae improved from 0.02036 to 0.01979, saving model to ../../saved_models/water_N_E_95.h5
Epoch 149/1000
126/126 - 3s - loss: 9.0739e-05 - mae: 0.0056 - val_loss: 7.1853e-04 - val_mae: 0.0207

Epoch 00149: val_mae did not improve from 0.01979
Epoch 150/1000
126/126 - 3s - loss: 8.9870e-05 - mae: 0.0056 - val_loss: 6.8393e-04 - val_mae: 0.0202

Epoch 00150: val_mae did not improve from 0.01979
Epoch 151/1000
126/126 - 3s - loss: 9.0042e-05 - mae: 0.0056 - val_loss: 7.1798e-04 - val_mae: 0.0207

Epoch 00151: val_mae did 

126/126 - 3s - loss: 8.3111e-05 - mae: 0.0053 - val_loss: 6.8401e-04 - val_mae: 0.0200

Epoch 00198: val_mae did not improve from 0.01943
Epoch 199/1000
126/126 - 3s - loss: 8.3613e-05 - mae: 0.0053 - val_loss: 6.7598e-04 - val_mae: 0.0199

Epoch 00199: val_mae did not improve from 0.01943
Epoch 200/1000
126/126 - 3s - loss: 8.3277e-05 - mae: 0.0053 - val_loss: 6.8196e-04 - val_mae: 0.0200

Epoch 00200: val_mae did not improve from 0.01943
Epoch 201/1000
126/126 - 3s - loss: 8.3740e-05 - mae: 0.0053 - val_loss: 6.7673e-04 - val_mae: 0.0198

Epoch 00201: val_mae did not improve from 0.01943
Epoch 202/1000
126/126 - 2s - loss: 8.2944e-05 - mae: 0.0053 - val_loss: 6.7365e-04 - val_mae: 0.0199

Epoch 00202: val_mae did not improve from 0.01943
Epoch 203/1000
126/126 - 3s - loss: 8.2795e-05 - mae: 0.0053 - val_loss: 6.3859e-04 - val_mae: 0.0193

Epoch 00203: val_mae improved from 0.01943 to 0.01932, saving model to ../../saved_models/water_N_E_95.h5
Epoch 204/1000
126/126 - 3s - loss: 8.314

126/126 - 3s - loss: 7.8929e-05 - mae: 0.0051 - val_loss: 7.0186e-04 - val_mae: 0.0200

Epoch 00251: val_mae did not improve from 0.01924
Epoch 252/1000
126/126 - 3s - loss: 7.8754e-05 - mae: 0.0051 - val_loss: 6.9934e-04 - val_mae: 0.0199

Epoch 00252: val_mae did not improve from 0.01924
Epoch 253/1000
126/126 - 3s - loss: 7.8832e-05 - mae: 0.0051 - val_loss: 6.7689e-04 - val_mae: 0.0197

Epoch 00253: val_mae did not improve from 0.01924
Epoch 254/1000
126/126 - 3s - loss: 7.9141e-05 - mae: 0.0052 - val_loss: 7.5212e-04 - val_mae: 0.0206

Epoch 00254: val_mae did not improve from 0.01924
Epoch 255/1000
126/126 - 3s - loss: 7.9092e-05 - mae: 0.0052 - val_loss: 6.8013e-04 - val_mae: 0.0197

Epoch 00255: val_mae did not improve from 0.01924
Epoch 256/1000
126/126 - 2s - loss: 7.8408e-05 - mae: 0.0051 - val_loss: 7.0388e-04 - val_mae: 0.0200

Epoch 00256: val_mae did not improve from 0.01924
Epoch 257/1000
126/126 - 3s - loss: 7.8534e-05 - mae: 0.0051 - val_loss: 7.1352e-04 - val_mae: 0.


Epoch 00304: val_mae did not improve from 0.01924
Epoch 305/1000
126/126 - 3s - loss: 7.5400e-05 - mae: 0.0050 - val_loss: 7.6893e-04 - val_mae: 0.0205

Epoch 00305: val_mae did not improve from 0.01924
Epoch 306/1000
126/126 - 3s - loss: 7.5600e-05 - mae: 0.0050 - val_loss: 7.0135e-04 - val_mae: 0.0198

Epoch 00306: val_mae did not improve from 0.01924
Epoch 307/1000
126/126 - 3s - loss: 7.5159e-05 - mae: 0.0050 - val_loss: 7.4087e-04 - val_mae: 0.0202

Epoch 00307: val_mae did not improve from 0.01924
Epoch 308/1000
126/126 - 3s - loss: 7.5134e-05 - mae: 0.0050 - val_loss: 7.8421e-04 - val_mae: 0.0207

Epoch 00308: val_mae did not improve from 0.01924
Epoch 309/1000
126/126 - 3s - loss: 7.5401e-05 - mae: 0.0050 - val_loss: 7.0769e-04 - val_mae: 0.0199

Epoch 00309: val_mae did not improve from 0.01924
Epoch 310/1000
126/126 - 3s - loss: 7.5040e-05 - mae: 0.0050 - val_loss: 7.0031e-04 - val_mae: 0.0197

Epoch 00310: val_mae did not improve from 0.01924
Epoch 311/1000
126/126 - 3s - l